# Setup ChromaDB

In [2]:
import chromadb

In [3]:

chroma_client = chromadb.Client()


Listar  las colecciones

In [56]:
chroma_client.list_collections()

Crear una colección de prueba

In [5]:
collection = chroma_client.create_collection(name="prueba")

In [9]:
# This allows us to create a client that connects to the server
collection = chroma_client.create_collection(name="my_collection_test")

In [16]:
chroma_client.list_collections()

Cargar texto en la colección

In [15]:
collection.add(
    documents=['Primer documento', 'Segundo documento', 'Tercer documento'],
    metadatas=[{"doc":'cine'}, {"doc":'musica'}, {"doc":'deporte'}],
    ids=['id1', 'id2', 'id3'],
)   

C:\Users\mavix\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:04<00:00, 20.6MiB/s]


Buscar texto 

In [20]:
result = collection.query(
    query_texts=['Primer documento'],
    # query_embeddings=None,
    # query_images=None,
    # query_uris=None,
    n_results=2,
)
result

{'ids': [['id1', 'id3']],
 'distances': [[0.0, 0.6515375971794128]],
 'metadatas': [[{'doc': 'cine'}, {'doc': 'deporte'}]],
 'embeddings': None,
 'documents': [['Primer documento', 'Tercer documento']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [ ]:
Embedding

In [21]:
from chromadb.utils import embedding_functions

Saber que modelo estamos usando por defecto

In [22]:
embedding_functions.DefaultEmbeddingFunction()

## Tambien podemos trabajar con nuestro propio modelo	

### Importamos las librerias necesarias para leer las variables de entorno

Nota: se debe crear un acrchivo .env en la raiz del proyecto con las siguientes variables
```
COHERE_API_KEY=xxxx

``` 
Necesitaras ejecutar los siguientes comandos para instalar las librerias necesarias
```bash
pip install python-dotenv
pip install cohere
pip install --upgrade langchain
pip install sentence_transformers
```

```python


### Descargando un modelo

In [48]:
# · Nos conectamos con huggingface para obtener el modelo de embedding
Sentence_embedding = embedding_functions.SentenceTransformerEmbeddingFunction(
    # model_name="paraphrase-MiniLM-L6-v2",
    model_name="sentence-transformers/paraphrase-Multilingual-MiniLM-L12-v2",
    # device = "cpu",
    # normalize_embeddings = False,

)

In [52]:
embedding_vector = Sentence_embedding(['La plaza del Zocalo es el centro de la Ciudad de México'])

In [54]:
len(embedding_vector), len(embedding_vector[0])

(1, 384)

In [74]:
collection.add(
    embeddings=embedding_vector,
    documents=['La plaza del Zocalo es el centro de la Ciudad de México'],
    metadatas=[{"doc":'turismo'}],
    ids=['id4'],
)

### Si querenmos usar otro modelos

In [57]:
new_collection = chroma_client.create_collection(
    name="prueba_embedding",
    embedding_function=Sentence_embedding,)

In [75]:
chroma_client.list_collections()

## Persistencia de BD

Persistencia de la base de datos en Local

Notas:
para poder tener la persistencia del RAG deberemos cargar el montar el docker de chromaDB
primero si estamos en windows deberemos instalar docker desktop 
y wls2 para poder montar el docker de chromaDB
abrimos ubuntu y ejecutamos el siguiente comando
```bash
docker pull chromadb/chroma
```
para montar el docker de chromaDB ejecutamos el siguiente comando

```bash
docker run -p 8000:8000 --name chroma chromadb/chroma
```
Si falla debemos ejecutar el siguiente comando

```bash
docker run --name chroma --env=CHROMA_HOST_PORT=8002 --workdir=/chroma -p 8002:8002 -d chromadb/chroma```

Posiblemente el error sea por los puertos que estan ocupados




In [1]:
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8002)


In [66]:
chroma_client.list_collections()

[]

In [2]:
collection = chroma_client.create_collection(name="prueba",get_or_create=True)

In [4]:
chroma_client.list_collections()

In [3]:
collection.add(
    embeddings=embedding_vector,
    documents=['La plaza del Zocalo es el centro de la Ciudad de México'],
    metadatas=[{"doc":'turismo'}],
    ids=['id4'],
)

NameError: name 'embedding_vector' is not defined

In [5]:
result = collection.query(
    query_texts=['plaza'],
    # query_embeddings=None,
    # query_images=None,
    # query_uris=None,
    n_results=2,
)
result

{'ids': [['id4']],
 'distances': [[21.46262966637728]],
 'embeddings': None,
 'metadatas': [[{'doc': 'turismo'}]],
 'documents': [['La plaza del Zocalo es el centro de la Ciudad de México']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [71]:
chroma_client.heartbeat()chroma_client.heartbeat()

1719426619733426414

### Persistencia en local con una ruta

In [6]:
pwd

'f:\\Proyectos\\PiConsulting\\challenge_rag_llm\\Notebooks'

In [4]:
client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")


In [ ]:
client.list_collections()

In [9]:
client.heartbeat()

1719427791110107600

In [5]:
client.list_collections()

[]

In [6]:
collection_local = client.create_collection(name="prueba_local",get_or_create=True)

In [7]:
client.list_collections()

In [8]:
collection_local.add(
    # embeddings=embedding_vector,
    documents=['La plaza del Zocalo es el centro de la Ciudad de México'],
    metadatas=[{"doc":'turismo'}],
    ids=['id4'],
)

In [15]:
client.list_collections()

## Crear embeddings con Cohere

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [25]:
COHERE_api_key: str = os.getenv("COHERE_API_KEY")

In [26]:
#openai_ef = embedding_functions.OpenAIEmbeddingFunction(model_name="text-embedding-ada-002")
cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key=COHERE_api_key,  model_name="large")

In [29]:
metadata_options = {
    "hnsw:space": "ip"  # You can change this to "ip" or "cosine" if needed
}


collection = chroma_client.get_or_create_collection(
    name="my_collection", metadata=metadata_options, embedding_function=cohere_ef)



In [30]:
sentence_embedding = collection.

AttributeError: 'Collection' object has no attribute 'embedding_function'

In [28]:
chroma_client.list_collections()

In [10]:
from docx import Document

# Abre el archivo .docx
doc = Document('F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx')

# Lee el contenido del documento
for paragraph in doc.paragraphs:
    print(paragraph.text)
    print('')


Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.

Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.

Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la 

In [19]:
import os 
from dotenv import load_dotenv
load_dotenv('./../../.env',verbose=True,override=True)
os.getenv("COHERE_API_KEY")


'5J8KjCW40CM4ee5608HcNCvQ1il8dHgORTKWq6zs'

In [21]:
load_dotenv(override=True)
os.getenv("COHERE_API_KEY")


'EVhu462XseHD8VrbEy7S82Vnxy0pkQKmEx2rARKY'